# Spark Lab 6: Use Spark SQL to Join and Aggregate Data (Solution)

In this lab, we will learn a few things:

- how to read json type (a specific kind with each row is a json-encoded record) data into Spark SQL
- how to use SQL functions to do some more advanced data manipulations (e.g. split).
- how to do Inner joins 
- how to do some reporting using windowing functions (more advanced, but optional).

## Dataset

We will use a few files distributed with `sparkdata.zip`. If you have not previously downloaded `sparkdata.zip`, you can download it from `http://idsdl.csom.umn.edu/c/share/sparkdata.zip` using `wget`. Alternatively, you can copy the URL in your browser and download it from there. 

- `loudacre/device.json`: list of devices
- `loudacre/webpage.json`: inventory of webpages
- `loudacre/websitehit.json`:  hits on webpage with device_id

## Step 1. Inspect the Data

First, you want to inspect the data so that you understand its format.

Use OS commands to view a sample of each json file

In [141]:
!head sparkdata/loudacre/webpage.json

In [142]:
!head sparkdata/loudacre/device.json

In [143]:
!head sparkdata/loudacre/websitehit.json

**Question**: 
- given the format, what is the best way to read these files?  
- Do we have consistent field names across tables?

**Answer**:
- It is best to use json reader (each row is a json-encoded record)
- The names are inconsistent, e.g. device_id and device_num

## Step 2. Read, Inspect, and Analyze `webpage`

First load `webpage.json` into a DataFrame called `webpage`

In [76]:
webpage = spark.read.json("sparkdata/loudacre/webpage.json")

Inspect the schema and first 20 rows. Fix any issue, if any, before you proceed.

**tip**: use `show(truncate=False)` to show long fields completely.

In [77]:
webpage.printSchema()

root
 |-- associated_files: string (nullable = true)
 |-- web_page_file_name: string (nullable = true)
 |-- web_page_num: long (nullable = true)



In [123]:
webpage.show(truncate=False)

+------------------------------------------+-------------------------------+------------+
|associated_files                          |web_page_file_name             |web_page_num|
+------------------------------------------+-------------------------------+------------+
|theme.css,code.js,sorrento_f00l.jpg       |sorrento_f00l_sales.html       |1           |
|theme.css,code.js,titanic_2100.jpg        |titanic_2100_sales.html        |2           |
|theme.css,code.js,meetoo_3.0.jpg          |meetoo_3.0_sales.html          |3           |
|theme.css,code.js,meetoo_3.1.jpg          |meetoo_3.1_sales.html          |4           |
|theme.css,code.js,ifruit_1.jpg            |ifruit_1_sales.html            |5           |
|theme.css,code.js,ifruit_3.jpg            |ifruit_3_sales.html            |6           |
|theme.css,code.js,ifruit_2.jpg            |ifruit_2_sales.html            |7           |
|theme.css,code.js,ifruit_5.jpg            |ifruit_5_sales.html            |8           |
|theme.css

You notice that the associated_files lists multiple files separated by commas. Next we want to list these files individually, such as:

```
+------------+-----------------+
|web_page_num|       assoc_file|
+------------+-----------------+
|           1|        theme.css|
|           1|          code.js|
|           1|sorrento_f00l.jpg|
|           2|        theme.css|
|           2|          code.js|
|           2| titanic_2100.jpg|
```

Achive the above goal (this helps, for example, you to run query on file hits).

- i.e. create a dataframe `page_files` with `web_page_num` and `assoc_file`

**Hint**: consider using Spark SQL functions to first split the field, then explode it.

In [79]:
import pyspark.sql.functions as f

In [80]:
page_files = webpage.select("web_page_num", \
             f.explode(f.split(webpage.associated_files,",")).alias("assoc_file"))

Verify what you obtain:

In [81]:
page_files.show()

+------------+-----------------+
|web_page_num|       assoc_file|
+------------+-----------------+
|           1|        theme.css|
|           1|          code.js|
|           1|sorrento_f00l.jpg|
|           2|        theme.css|
|           2|          code.js|
|           2| titanic_2100.jpg|
|           3|        theme.css|
|           3|          code.js|
|           3|   meetoo_3.0.jpg|
|           4|        theme.css|
|           4|          code.js|
|           4|   meetoo_3.1.jpg|
|           5|        theme.css|
|           5|          code.js|
|           5|     ifruit_1.jpg|
|           6|        theme.css|
|           6|          code.js|
|           6|     ifruit_3.jpg|
|           7|        theme.css|
|           7|          code.js|
+------------+-----------------+
only showing top 20 rows



To practice JOIN with Spark, we ask you to join the `webpage` and `page_files`


In [82]:
webpage_files = webpage.join(page_files, "web_page_num") \
    .select("web_page_num","web_page_file_name",page_files.assoc_file)

Verify your results

In [83]:
webpage_files.show()

+------------+--------------------+-----------------+
|web_page_num|  web_page_file_name|             file|
+------------+--------------------+-----------------+
|           1|sorrento_f00l_sal...|sorrento_f00l.jpg|
|           1|sorrento_f00l_sal...|          code.js|
|           1|sorrento_f00l_sal...|        theme.css|
|           2|titanic_2100_sale...| titanic_2100.jpg|
|           2|titanic_2100_sale...|          code.js|
|           2|titanic_2100_sale...|        theme.css|
|           3|meetoo_3.0_sales....|   meetoo_3.0.jpg|
|           3|meetoo_3.0_sales....|          code.js|
|           3|meetoo_3.0_sales....|        theme.css|
|           4|meetoo_3.1_sales....|   meetoo_3.1.jpg|
|           4|meetoo_3.1_sales....|          code.js|
|           4|meetoo_3.1_sales....|        theme.css|
|           5| ifruit_1_sales.html|     ifruit_1.jpg|
|           5| ifruit_1_sales.html|          code.js|
|           5| ifruit_1_sales.html|        theme.css|
|           6| ifruit_3_sale

## Step 3: Find top most-used devices for each page (optional, more challenging)

When a user visits a page using a device, this gets saved to `websitehit`. We want to analyze **for each webpage, what are the top 2 devices used for visiting this page**? 

This is most conveniently accomplished using Spark SQL's window functions (in particular its `rank()` function, because if you can get the rank of records by # of hits per device, then you can filter the dataset by rank to show just the first two). if you need refresher of window functions, you can visit [https://databricks.com/blog/2015/07/15/introducing-window-functions-in-spark-sql.html](https://databricks.com/blog/2015/07/15/introducing-window-functions-in-spark-sql.html).


But before you can get the top ranks, you may first need to aggregate the data to get the # of hits.

In the end, we want you to show something like this (some table joins are needed, think about when you should do joins):

```
+-------------------------------+-------------+----+
|web_page_file_name             |device_name  |hits|
+-------------------------------+-------------+----+
|sorrento_f30l_sales.html       |Sorrento F41L|125 |
|sorrento_f30l_sales.html       |Titanic 1100 |68  |
|sorrento_f41l_sales.html       |Sorrento F41L|116 |
|sorrento_f41l_sales.html       |Titanic 1000 |64  |
|ronin_novelty_note_4_sales.html|Sorrento F41L|123 |
|ronin_novelty_note_4_sales.html|Titanic 1100 |63  |
|sorrento_f24l_sales.html       |Sorrento F41L|122 |
|sorrento_f24l_sales.html       |Titanic 1100 |63  |
```


In [85]:
hits = spark.read.json("sparkdata/loudacre/websitehit.json")

In [86]:
devices = spark.read.json("sparkdata/loudacre/device.json")

In [87]:
hits.show()

+---------+-------------------+-----------+
|device_id|           hit_time|web_page_id|
+---------+-------------------+-----------+
|       34|2013-09-15 02:26:31|         25|
|       33|2013-09-15 21:08:13|         14|
|        2|2013-09-15 02:33:57|          8|
|       10|2013-09-15 11:41:32|         19|
|       15|2013-09-15 02:42:22|         46|
|       20|2013-09-15 23:31:10|         22|
|       33|2013-09-15 10:59:42|         45|
|        1|2013-09-15 03:22:37|         31|
|        6|2013-09-15 09:04:35|         41|
|       47|2013-09-15 07:05:25|         32|
|       38|2013-09-15 08:11:10|         34|
|       38|2013-09-15 07:53:58|         33|
|       34|2013-09-15 07:19:51|         28|
|       10|2013-09-15 13:15:15|         50|
|       35|2013-09-15 16:59:46|          5|
|       14|2013-09-15 00:16:56|          8|
|       44|2013-09-15 00:19:40|         40|
|        6|2013-09-15 21:29:20|         19|
|       26|2013-09-15 19:03:05|         29|
|       33|2013-09-15 02:56:19| 

In [113]:
devices.show()

+--------------------+----------+-----------+-------------------+
|         device_name|device_num|device_type|         release_dt|
+--------------------+----------+-----------+-------------------+
|       Sorrento F00L|         1|      phone|2008-10-21 00:00:00|
|        Titanic 2100|         2|      phone|2010-04-19 00:00:00|
|          MeeToo 3.0|         3|      phone|2011-02-18 00:00:00|
|          MeeToo 3.1|         4|      phone|2011-09-21 00:00:00|
|            iFruit 1|         5|      phone|2008-10-21 00:00:00|
|            iFruit 3|         6|      phone|2011-11-02 00:00:00|
|            iFruit 2|         7|      phone|2010-05-20 00:00:00|
|            iFruit 5|         8|      phone|2013-07-02 00:00:00|
|        Titanic 1000|         9|      phone|2008-10-21 00:00:00|
|          MeeToo 1.0|        10|      phone|2008-10-21 00:00:00|
|       Sorrento F21L|        11|      phone|2011-02-28 00:00:00|
|            iFruit 4|        12|      phone|2012-10-25 00:00:00|
|       So

In [88]:
hits.createOrReplaceTempView("hits")

In [127]:
stat = spark.sql("""
    select web_page_id, device_id, count(*) as hits 
    from hits 
    group by web_page_id, device_id 
    order by web_page_id, count(*) desc
""").cache()

In [128]:
stat.show()

+-----------+---------+----+
|web_page_id|device_id|hits|
+-----------+---------+----+
|          1|       29| 120|
|          1|        1|  62|
|          1|       38|  61|
|          1|       43|  56|
|          1|       10|  53|
|          1|        9|  51|
|          1|       27|  43|
|          1|       23|  42|
|          1|        5|  42|
|          1|        7|  42|
|          1|       15|  38|
|          1|        2|  38|
|          1|       34|  38|
|          1|       47|  37|
|          1|       33|  34|
|          1|       44|  33|
|          1|       21|  32|
|          1|       45|  30|
|          1|       14|  29|
|          1|        3|  27|
+-----------+---------+----+
only showing top 20 rows



In [129]:
from pyspark.sql.window import Window

wind = Window.partitionBy(stat.web_page_id).orderBy(stat.hits.desc())


In [136]:
top2 = stat.select("web_page_id","device_id","hits", \
    f.dense_rank().over(wind).alias("r")).where("r<3")


In [137]:
top2.show()

+-----------+---------+----+---+
|web_page_id|device_id|hits|  r|
+-----------+---------+----+---+
|         26|       29| 125|  1|
|         26|       38|  68|  2|
|         29|       29| 116|  1|
|         29|        9|  64|  2|
|         19|       29| 123|  1|
|         19|       38|  63|  2|
|         22|       29| 122|  1|
|         22|       38|  63|  2|
|          7|       29| 121|  1|
|          7|        1|  67|  2|
|         34|       29| 122|  1|
|         34|        1|  56|  2|
|         34|       38|  56|  2|
|         34|        5|  56|  2|
|         50|       29| 128|  1|
|         50|       10|  64|  2|
|         32|       29| 145|  1|
|         32|       38|  52|  2|
|         43|       29| 113|  1|
|         43|       38|  61|  2|
+-----------+---------+----+---+
only showing top 20 rows



how we try to join tables to proper names.

In [138]:
top2_revised = top2.join(devices, top2.device_id == devices.device_num) \
    .join(webpage, top2.web_page_id == webpage.web_page_num) \
    .select(webpage.web_page_file_name, devices.device_name, top2.hits)

In [140]:
top2_revised.show(truncate=False)

+-------------------------------+-------------+----+
|web_page_file_name             |device_name  |hits|
+-------------------------------+-------------+----+
|sorrento_f30l_sales.html       |Sorrento F41L|125 |
|sorrento_f30l_sales.html       |Titanic 1100 |68  |
|sorrento_f41l_sales.html       |Sorrento F41L|116 |
|sorrento_f41l_sales.html       |Titanic 1000 |64  |
|ronin_novelty_note_4_sales.html|Sorrento F41L|123 |
|ronin_novelty_note_4_sales.html|Titanic 1100 |63  |
|sorrento_f24l_sales.html       |Sorrento F41L|122 |
|sorrento_f24l_sales.html       |Titanic 1100 |63  |
|ifruit_2_sales.html            |Sorrento F41L|121 |
|ifruit_2_sales.html            |Sorrento F00L|67  |
|titanic_2000_sales.html        |Sorrento F41L|122 |
|titanic_2000_sales.html        |Sorrento F00L|56  |
|titanic_2000_sales.html        |Titanic 1100 |56  |
|titanic_2000_sales.html        |iFruit 1     |56  |
|ifruit_5a_sales.html           |Sorrento F41L|128 |
|ifruit_5a_sales.html           |MeeToo 1.0   